## 1. Import dependencies

In [1]:
import pandas as pd
# !pip install openpyxl
import openpyxl
from sqlalchemy import create_engine
from db_conn import DB_conn

## 2. Retrieve Data from the excel file

In [2]:
excel_file = 'Data_Tables_LGA_Criminal_Incidents_Year_Ending_March_2022.xlsx'
all_sheets = pd.read_excel(excel_file, sheet_name=None)
sheets = all_sheets.keys()

for sheet_name in sheets:
    sheet = pd.read_excel(excel_file, sheet_name=sheet_name)
    sheet.to_csv("%s.csv" % sheet_name, index=False)

Contents	
Table 01

* Criminal incidents and rate per 100,000 population by police region and local government area - April 2012 to March 2022

Table 02	

* Criminal incidents and rate per 100,000 population by principal offence, local government area and police service area - April 2012 to March 2022

Table 03	

* Criminal incidents by principal offence, local government area and postcode or suburb/town - April 2012 to March 2022

Table 04	

* Criminal incidents by location type and local government area - April 2012 to March 2022

Table 05	

* Criminal incidents by charge status and local government area - April 2012 to March 2022
	
### Data exclusions	
Table 01 

* excludes criminal incidents where the geographic location is unknown. For further information of these geographic locations please refer to the Explanatory Notes and Glossary on the website.	

Tables 02 - 05 

* excludes criminal incidents at Justice institutions and immigration facilities, Unincorporated Victoria and where the geographic location is unknown. For further information of these geographic locations please refer to the Explanatory Notes and Glossary on the website.	
	
Notes	
Recorded crime statistics are based on data extracted by Victoria police on the 18th day after the reference period, and are subject to movement between releases. For more information about how statistics are compiled, refer to the Explanatory notes on the CSA website.	
In order to maintain confidentiality, sensitive offence counts for subdivisions 'A10 Homicide and related offences' and 'A30 Sexual offences' with a value of 3 or less are given a value of 2 to calculate totals.	
Rates are based on populations provided by the Australian Bureau of Statistics (ABS). The most recent year of data was not available from the ABS in time for it to be used to calculate current year rates. The CSA uses estimates created by the Victorian State Government 'Victoria in Futures' report. 	
This work is licenced under a Creative Commons Attribution 4.0 International License. When reporting this data you must attribute the Crime Statistics Agency (or CSA) as the source.

## Loop thru excel file to convert various worksheets to csv files

In [3]:
#Create list of dataframes from csv files
file_list = []
for i in range(1,7):
    file_list.append(f"Table 0{i}.csv")

df_list = [pd.read_csv(file) for file in file_list]
file_list

['Table 01.csv',
 'Table 02.csv',
 'Table 03.csv',
 'Table 04.csv',
 'Table 05.csv',
 'Table 06.csv']

### Create police_region_df from Table1 which is high level data, starts from Police Region
(PR, a police regions have a one to many relationshgip with Police Service Area and local Government Areas )

In [35]:
# Exract rows that are greater than year 2018 (use 4 years data, 2019-2022) 
# and drop column 'Year ending' as the crime tables are created on March (no date details) every year.
police_region_df = df_list[0].loc[df_list[0].Year > 2018].drop('Year ending',axis=1)
police_region_df["Local Government Area"] = police_region_df["Local Government Area"].str.strip()

# Drop Total(summary) rows
print(f"Before Total row drop: {police_region_df.shape}")
police_region_df = police_region_df.drop(police_region_df[(police_region_df['Local Government Area'] == 'Total')].index)
print(f"After Total row drop: {police_region_df.shape}")
police_region_df.head()


Before Total row drop: (348, 5)
After Total row drop: (324, 5)


,Year,Police Region,Local Government Area,Incidents Recorded,"Rate per 100,000 population"
0,2022,1 North West Metro,Banyule,5244,4086.865525
1,2022,1 North West Metro,Brimbank,12330,6200.959480
2,2022,1 North West Metro,Darebin,9276,5751.475727
3,2022,1 North West Metro,Hobsons Bay,4599,4804.896128
4,2022,1 North West Metro,Hume,12196,4919.684615


In [11]:
# police_region_df.dtypes

### Create police_division_df from Table2 which is in Police Service Area (PSA) and Local Government Area (LGA), 
breaks down to different crime/offence and subtype types

In [32]:
# Due to Heroku free data capacity allowance constraint, we use data rows of year 2022.
police_division_df = df_list[1].loc[df_list[1].Year > 2021].drop('Year ending',axis=1)
print(f"table shape: {police_division_df.shape}")
police_division_df.head()

table shape: (5017, 9)


,Year,Police Service Area,Local Government Area,Offence Division,Offence Subdivision,Offence Subgroup,Incidents Recorded,"PSA Rate per 100,000 population","LGA Rate per 100,000 population"
0,2022,Ballarat,Ballarat,A Crimes against the person,A10 Homicide and related offences,A10 Homicide and related offences,5,4.059101,4.328089
1,2022,Ballarat,Ballarat,A Crimes against the person,A20 Assault and related offences,A211 FV Serious assault,120,97.418429,103.874128
2,2022,Ballarat,Ballarat,A Crimes against the person,A20 Assault and related offences,A212 Non-FV Serious assault,147,119.337575,127.245807
3,2022,Ballarat,Ballarat,A Crimes against the person,A20 Assault and related offences,"A22 Assault police, emergency services or othe...",53,43.026473,45.877740
4,2022,Ballarat,Ballarat,A Crimes against the person,A20 Assault and related offences,A231 FV Common assault,222,180.224093,192.167137


In [13]:
# police_division_df.dtypes 

### Merged Police Region column on Local Government Area Column to create a more complete data
set to enable consisent filtering for data visualisati0n 

In [33]:
police_region2022_df = ''
police_region2022_df = df_list[0].loc[df_list[0].Year > 2021].drop('Year ending',axis=1)
police_region2022_df["Local Government Area"] = police_region2022_df["Local Government Area"].str.strip() # must strip off the first blank character
print(f"table shape: {police_region2022_df.shape}")

table shape: (87, 5)


In [25]:
police_region2022_df['Local Government Area'].unique()

array(['Banyule', 'Brimbank', 'Darebin', 'Hobsons Bay', 'Hume',
       'Maribyrnong', 'Melbourne', 'Melton', 'Moonee Valley', 'Moreland',
       'Nillumbik', 'Whittlesea', 'Wyndham', 'Yarra', 'Total', 'Alpine',
       'Bass Coast', 'Baw Baw', 'Benalla', 'Boroondara', 'East Gippsland',
       'Greater Shepparton', 'Indigo', 'Knox', 'Latrobe', 'Manningham',
       'Mansfield', 'Maroondah', 'Mitchell', 'Moira', 'Monash',
       'Murrindindi', 'South Gippsland', 'Strathbogie', 'Towong',
       'Wangaratta', 'Wellington', 'Whitehorse', 'Wodonga',
       'Yarra Ranges', 'Bayside', 'Cardinia', 'Casey', 'Frankston',
       'Glen Eira', 'Greater Dandenong', 'Kingston',
       'Mornington Peninsula', 'Port Phillip', 'Stonnington', 'Ararat',
       'Ballarat', 'Buloke', 'Campaspe', 'Central Goldfields',
       'Colac-Otway', 'Corangamite', 'Gannawarra', 'Glenelg',
       'Golden Plains', 'Greater Bendigo', 'Greater Geelong', 'Hepburn',
       'Hindmarsh', 'Horsham', 'Loddon', 'Macedon Ranges', 'M

In [34]:
police_division_df=pd.merge(police_division_df, police_region2022_df[["Police Region","Local Government Area"]],
           how="left", on="Local Government Area")
police_division_df=police_division_df[police_division_df["Year"]> 2021]
print(f"table shape: {police_division_df.shape}")
police_division_df.head()     

table shape: (5017, 10)


,Year,Police Service Area,Local Government Area,Offence Division,Offence Subdivision,Offence Subgroup,Incidents Recorded,"PSA Rate per 100,000 population","LGA Rate per 100,000 population",Police Region
0,2022,Ballarat,Ballarat,A Crimes against the person,A10 Homicide and related offences,A10 Homicide and related offences,5,4.059101,4.328089,4 Western
1,2022,Ballarat,Ballarat,A Crimes against the person,A20 Assault and related offences,A211 FV Serious assault,120,97.418429,103.874128,4 Western
2,2022,Ballarat,Ballarat,A Crimes against the person,A20 Assault and related offences,A212 Non-FV Serious assault,147,119.337575,127.245807,4 Western
3,2022,Ballarat,Ballarat,A Crimes against the person,A20 Assault and related offences,"A22 Assault police, emergency services or othe...",53,43.026473,45.877740,4 Western
4,2022,Ballarat,Ballarat,A Crimes against the person,A20 Assault and related offences,A231 FV Common assault,222,180.224093,192.167137,4 Western


### Create charge_status_df from Table5 which is in Local Government Region with 3 charge Status

In [29]:
charge_status_df = df_list[4].loc[df_list[4].Year > 2018].drop('Year ending',axis=1)
print(f"table shape: {charge_status_df.shape}")
charge_status_df.head()

table shape: (948, 4)


,Year,Local Government Area,Charge Status,Incidents Recorded
0,2022,Alpine,Charges laid,149
1,2022,Alpine,No charges laid,114
2,2022,Alpine,Unsolved,108
3,2022,Ararat,Charges laid,467
4,2022,Ararat,No charges laid,217


In [9]:
charge_status_df.dtypes 

Year                      int64
Local Government Area    object
Charge Status            object
Incidents Recorded        int64
dtype: object

### Create offence_summary_df for Local Government Region base on different crime types

In [28]:
offence_summary_df = df_list[5]
offence_summary_df.head()

,year,lga_name,a_crime_vs_person,b_property_deception,c_drug_offence,d_public_order_security,e_justice_offence,f_other_offence,(blank)
0,2019,Alpine,114,161,26.0,48,50,0,NaN
1,2019,Ararat,223,376,65.0,81,165,2,NaN
2,2019,Ballarat,1372,5061,204.0,372,1105,15,NaN
3,2019,Banyule,957,4063,223.0,309,1442,11,NaN
4,2019,Bass Coast,376,992,78.0,119,216,2,NaN


In [10]:
offence_summary_df.dtypes 

Year,Local Government Area,A Crimes against the person,B Property and deception offences,C Drug offences,D Public order and security offences,E Justice procedures offences,F Other offences,(blank)    object
dtype: object

## 3. Rename data frame columns to have more unified names

In [36]:
police_region_df.rename(columns = {'Year' : 'year', 'Police Region' : 'region_name', 
            'Local Government Area' : 'lga_name', 'Incidents Recorded' : 'incident_count',
            'Rate per 100,000 population' : 'rate_per_100000pop'}, inplace = True)  
police_region_df.head()         

,year,region_name,lga_name,incident_count,rate_per_100000pop
0,2022,1 North West Metro,Banyule,5244,4086.865525
1,2022,1 North West Metro,Brimbank,12330,6200.959480
2,2022,1 North West Metro,Darebin,9276,5751.475727
3,2022,1 North West Metro,Hobsons Bay,4599,4804.896128
4,2022,1 North West Metro,Hume,12196,4919.684615


In [37]:
police_division_df.rename(columns = {'Year' : 'year', 'Police Service Area' : 'psa_name',
            'Local Government Area' : 'lga_name', 'Offence Division' : 'offence_div_name',
            'Offence Subdivision' : 'offence_subdiv_name', 'Offence Subgroup' : 'offence_subgrp_name',
            'Incidents Recorded' : 'incident_count', 'PSA Rate per 100,000 population' : 'psa_rate_per_100000pop',
            'LGA Rate per 100,000 population' : 'lga_rate_per_100000pop', 'Police Region' : 'region_name'}, inplace = True)
police_division_df.head() 

,year,psa_name,lga_name,offence_div_name,offence_subdiv_name,offence_subgrp_name,incident_count,psa_rate_per_100000pop,lga_rate_per_100000pop,region_name
0,2022,Ballarat,Ballarat,A Crimes against the person,A10 Homicide and related offences,A10 Homicide and related offences,5,4.059101,4.328089,4 Western
1,2022,Ballarat,Ballarat,A Crimes against the person,A20 Assault and related offences,A211 FV Serious assault,120,97.418429,103.874128,4 Western
2,2022,Ballarat,Ballarat,A Crimes against the person,A20 Assault and related offences,A212 Non-FV Serious assault,147,119.337575,127.245807,4 Western
3,2022,Ballarat,Ballarat,A Crimes against the person,A20 Assault and related offences,"A22 Assault police, emergency services or othe...",53,43.026473,45.877740,4 Western
4,2022,Ballarat,Ballarat,A Crimes against the person,A20 Assault and related offences,A231 FV Common assault,222,180.224093,192.167137,4 Western


In [38]:
charge_status_df.rename(columns = {'Year' : 'year', 'Local Government Area' : 'lga_name', 
            'Charge Status' : 'charge_status_name', 'Incidents Recorded' : 'incident_count'}, inplace = True)
charge_status_df.head() 

,year,lga_name,charge_status_name,incident_count
0,2022,Alpine,Charges laid,149
1,2022,Alpine,No charges laid,114
2,2022,Alpine,Unsolved,108
3,2022,Ararat,Charges laid,467
4,2022,Ararat,No charges laid,217


In [39]:
offence_summary_df.rename(columns = {'Year' : 'year', 'Local Government Area' : 'lga_name',
            'A Crimes against the person' : 'a_crime_vs_person', 'B Property and deception offences' : 'b_property_deception',
            'C Drug offences' : 'c_drug_offence', 'D Public order and security offences' : 'd_public_order_security',
            'E Justice procedures offences' : 'e_justice_offence', 'F Other offences' : 'f_other_offence'}, inplace = True)

offence_summary_df = offence_summary_df.drop('(blank)',axis=1)
offence_summary_df.head()            

,year,lga_name,a_crime_vs_person,b_property_deception,c_drug_offence,d_public_order_security,e_justice_offence,f_other_offence
0,2019,Alpine,114,161,26.0,48,50,0
1,2019,Ararat,223,376,65.0,81,165,2
2,2019,Ballarat,1372,5061,204.0,372,1105,15
3,2019,Banyule,957,4063,223.0,309,1442,11
4,2019,Bass Coast,376,992,78.0,119,216,2


## 4. Export dataframes to csv files for checking data quality

In [40]:
police_region_df.to_csv('outputs/police_region_df.csv',index=0)
police_division_df.to_csv('outputs/police_division_df.csv',index=0)
charge_status_df.to_csv('outputs/charge_status_df.csv',index=0)
offence_summary_df.to_csv('outputs/offence_summary_df.csv',index=0)

## 5. Connect to database 

In [41]:
engine = create_engine (DB_conn)


In [42]:
engine.table_names()

c:\Users\youpe.DESKTOP-UVNLIE6\anaconda3\envs\bootcamp\lib\site-packages\ipykernel_launcher.py:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  """Entry point for launching an IPython kernel.


['lga_charge_status', 'lga_offence_summary']

In [43]:
police_region_df.to_sql(name='region_incident', con=engine, if_exists='replace', index=False)

In [44]:
print(pd.read_sql_query('select count(*) from region_incident', con=engine))
pd.read_sql_query('select * from region_incident', con=engine).head()

   count
0    324


,year,region_name,lga_name,incident_count,rate_per_100000pop
0,2022,1 North West Metro,Banyule,5244,4086.865525
1,2022,1 North West Metro,Brimbank,12330,6200.959480
2,2022,1 North West Metro,Darebin,9276,5751.475727
3,2022,1 North West Metro,Hobsons Bay,4599,4804.896128
4,2022,1 North West Metro,Hume,12196,4919.684615


In [46]:
police_division_df.to_sql(name='lga_offence', con=engine, if_exists='replace', index=False)

In [47]:
print(pd.read_sql_query('select count(*) from lga_offence', con=engine))
pd.read_sql_query('select * from lga_offence', con=engine).head()

   count
0   5017


,year,psa_name,lga_name,offence_div_name,offence_subdiv_name,offence_subgrp_name,incident_count,psa_rate_per_100000pop,lga_rate_per_100000pop,region_name
0,2022,Ballarat,Ballarat,A Crimes against the person,A10 Homicide and related offences,A10 Homicide and related offences,5,4.059101,4.328089,4 Western
1,2022,Ballarat,Ballarat,A Crimes against the person,A20 Assault and related offences,A211 FV Serious assault,120,97.418429,103.874128,4 Western
2,2022,Ballarat,Ballarat,A Crimes against the person,A20 Assault and related offences,A212 Non-FV Serious assault,147,119.337575,127.245807,4 Western
3,2022,Ballarat,Ballarat,A Crimes against the person,A20 Assault and related offences,"A22 Assault police, emergency services or othe...",53,43.026473,45.877740,4 Western
4,2022,Ballarat,Ballarat,A Crimes against the person,A20 Assault and related offences,A231 FV Common assault,222,180.224093,192.167137,4 Western


In [48]:
charge_status_df.to_sql(name='lga_charge_status', con=engine, if_exists='replace', index=False)

In [79]:
print(pd.read_sql_query('select count(*) from lga_charge_status', con=engine))
pd.read_sql_query('select * from lga_charge_status', con=engine).head()

   count
0    948


,year,lga_name,charge_status_name,incident_count
0,2022,Alpine,Charges laid,149
1,2022,Alpine,No charges laid,114
2,2022,Alpine,Unsolved,108
3,2022,Ararat,Charges laid,467
4,2022,Ararat,No charges laid,217


In [80]:
offence_summary_df.to_sql(name='lga_offence_summary', con=engine, if_exists='replace', index=False)
print(pd.read_sql_query('select count(*) from lga_offence_summary', con=engine))
pd.read_sql_query('select * from lga_offence_summary', con=engine).head()

In [81]:
print(pd.read_sql_query('select count(*) from lga_offence_summary', con=engine))
pd.read_sql_query('select * from lga_offence_summary', con=engine).head()

   count
0    316


,year,lga_name,a_crime_vs_person,b_property_deception,c_drug_offence,d_public_order_security,e_justice_offence,f_other_offence
0,2019,Alpine,114,161,26.0,48,50,0
1,2019,Ararat,223,376,65.0,81,165,2
2,2019,Ballarat,1372,5061,204.0,372,1105,15
3,2019,Banyule,957,4063,223.0,309,1442,11
4,2019,Bass Coast,376,992,78.0,119,216,2


In [82]:
# Check database table's availability
engine.table_names()

c:\Users\youpe.DESKTOP-UVNLIE6\anaconda3\envs\bootcamp\lib\site-packages\ipykernel_launcher.py:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  """Entry point for launching an IPython kernel.


['region_incident', 'lga_offence', 'lga_charge_status', 'lga_offence_summary']